# In-class Assignment 1

This assignment will first cover regular expressions, a.k.a RegEx. After that, we will make use of them to tokenize and normalize two different types of text. You can always check chapter 2 to make sure what operators you need for a specific task.

Let's start with some RegEx exercises!

## 1. Regular Expressions

We will use data from Twitter for this section:

In [4]:
twitter_file = 'twitter.txt'
!head -5 {twitter_file}

Lets have a date me and you for ever and ever 😍😘 #NeverAlone
ni el header he hecho....
va tener un hijo...confrsion es mioesta urbanisacion es enorme...
deseo acabar con todo...
@NEWdanicsierra em que?


#### 1.1. Using grep, find all the tweets (lines) that start with capital letters and either have a twitter username (@username) or a hashtag at the end.

**Hint**: you should get 26 lines. Use the pipe operator to count the lines: ``!grep ... | wc -l``

In [5]:
ex11 = !grep -E "^[A-Z].*(#[a-zA-Z0-9_]+|@[a-zA-Z0-9_]+)$" {twitter_file}
ex11

['Lets have a date me and you for ever and ever 😍😘 #NeverAlone',
 'Me encantaria que me vieras, de la manera que yo te veo #OneDay',
 "Going asleep with papi's music on skype, even thoe our call hang up #Priceless #DistanceDontMatter",
 'Que rico me duele mi cuerpo #goodexercise',
 'La conversación de precol mas vale que continue! @mememecaigo @PattyB_14  @salim_rosa',
 'Ya sabes mi bro arjona me gusta 😊😏 @Abnerdm',
 'I feel so sad 😔😞 #GodPleaseGuideMe',
 'Movie is about to begin #SonOfGod',
 'I hate the fact people get into my room and stuff start missing 😔😁😩 #WTH',
 'FULL RIDE :o #PARY',
 'I got 96 in my test #purocerebro',
 'Nike is making some pretty kiks #ilovethem',
 'DONT LIE TO ME ❌🚫😁 #backofme',
 'That sucks! But I like #MatthewMcConaughey',
 'I wanna go to sleep but I gotta see #Leo win that first Oscar #Oscars',
 'I take naps serious ..digo 15 mins. me le anto 3 horas despues-.- #always',
 "I'm always showing off @ReguloCaro 's daughter as if she's my own haha. #cutestkidevv

#### 1.2. Do the same exercise but using Python this time. 

**Hint**: the regular expression should be the same.

In [6]:
import re
import string

def read_file(file):
    return [l.rstrip() for l in open(file)]

def ex12(file):
    """Return the lines that match the regex pattern you did in the previous exercise"""

    """
    lines = read_file(file)
    list = []
    for i in range(len(lines)):
        r = re.compile('^[A-Z].*(#[a-zA-Z0-9_]+|@[a-zA-Z0-9_]+)$')
        j = r.findall(lines[i])
        if len(j) != 0:
             list.append(lines[i])
    return list
    """
    pattern = r'^[A-Z].*(@[a-zA-Z0-9_]+|#[A-Za-z0-9_]+)$'
    return [l for l in read_file(file) if re.findall(pattern, l)]
    print(len(ex12_sol(twitter_file)))
ex12(twitter_file)

['Lets have a date me and you for ever and ever 😍😘 #NeverAlone',
 'Me encantaria que me vieras, de la manera que yo te veo #OneDay',
 "Going asleep with papi's music on skype, even thoe our call hang up #Priceless #DistanceDontMatter",
 'Que rico me duele mi cuerpo #goodexercise',
 'La conversación de precol mas vale que continue! @mememecaigo @PattyB_14  @salim_rosa',
 'Ya sabes mi bro arjona me gusta 😊😏 @Abnerdm',
 'I feel so sad 😔😞 #GodPleaseGuideMe',
 'Movie is about to begin #SonOfGod',
 'I hate the fact people get into my room and stuff start missing 😔😁😩 #WTH',
 'FULL RIDE :o #PARY',
 'I got 96 in my test #purocerebro',
 'Nike is making some pretty kiks #ilovethem',
 'DONT LIE TO ME ❌🚫😁 #backofme',
 'That sucks! But I like #MatthewMcConaughey',
 'I wanna go to sleep but I gotta see #Leo win that first Oscar #Oscars',
 'I take naps serious ..digo 15 mins. me le anto 3 horas despues-.- #always',
 "I'm always showing off @ReguloCaro 's daughter as if she's my own haha. #cutestkidevv

#### 1.3. Find all the tweets that contains emojis

**Hint**: Use the following unicode block: `x1F000 - x2F000`.

If we were to search using grep, we would need to keep in mind that the file is written in UTF-8, and as such we need to use the hexadecimal values of the bytes in the regex pattern (you can use this link to see the conversion: https://r12a.github.io/apps/conversion/). For our unicode block we would have the following:
* `x1F000 -> F0 9F 80 80`
* `x2F000 -> F0 AF 80 80`

The command to grep a specific unicode in bytes would be like this:

In [7]:
!grep ""$'\xF0\x9F\x98\xB3'"" {twitter_file}

“@itseffinjas: I'm eating like I'm pregnant 😳” Maybe u is


Let's do the search for all the emojis but this time using Python (you won't need the byte representation this time)

In [12]:
def ex13(file):
    """Return the lines that contain emojis"""
    lines = read_file(file)
    inblock = lambda c:int('01F000', 16) <= ord(c) <= int('02F000',16)
    return [ l for l in lines if any(map(inblock,1))]


## 2. Tokenization

Tokenization can get complicated with a lot of edge cases. This time, we will just implement a toy tokenization function. Your function need to consider the following cases:
* Split by spaces, tabs, and enters:
    * `'  this is a tweet   ' -> 'this', 'is', 'a', 'tweet'`
* Split by emojis (keeping them):
    * `haha 😂😂! -> 'haha', '😂😂', '!'`
* Keep hashtags intact (i.e., do not split inside a hashtag):
    * `#don'tSplit `
* Separate punctuation marks from the text as individual tokens:
    * `'hey!' -> 'hey', '!'`
* Contractions need to be separated based on the apostrophe (without removing it):
    * `"that's fine" -> 'that', "'s", 'fine'`
    * `"don't do that" -> 'do', "n't", 'do', 'that'`

For contractions, consider the following ending cases:
* `'s`
* `n't`
* `'ll`
* `'m`
* `'ve`
* `'d`
* `'re`
* `'all`

**Hint**: Split using a cascade approach. That is, do not try to cover all the cases at once —divide and conquer!

In [7]:
def tokenize(sentence):
    """Return the tokens of the sentence"""
    tokens = sentence.strip().split()

    i = []
    for token in tokens:
        if not re.research("^[@#]", token):
            if re.search("[!?.,;(...):]", token):
              k = re.split("(!|?|.|;|...)")
            for eachone in k:
                h = re.split("\w+(?=n't)")
                j.extend(smallest_tokens)
        else:
            j.append(token)
    return tokens
def ex2(file):
    return [tokenize(l) for l in read_file(file)]
ex2(twitter_file)

AttributeError: module 're' has no attribute 'research'

In [1]:
res = !grep "["$'\xF0\x9F\x80\x80'"-"$'\xF0\xAF\x80\x80'"]" twitter.txt 

print(len(res))
res[:10]

1


['grep: Invalid collation character']

## 3. Normalization

Based on the previous tokenization, we will perform a normalization step. To keep it simple, we will only handle the case in which a sequence of letters has a repeated pattern. Look at the following examples:
* `hahahahaha -> haha`
* `cooooool -> cool`
* `!?!?!?!!!! -> !?!?!!`
* `!!!!!????? -> !!??`

**Hint:** Note that the reapeated patterns are reduce to 2 ocurrences

In [7]:
def ex3(tokenized_tweets):
    """Return a list of list where the inner items are the normalized tokens of each tweet"""

    #

    return []

## Grader

In [2]:
def grade_ex11():
    solution = [60, 63, 98, 41, 85, 39, 43, 34, 33, 133, 72, 18, 32, 58, 42, 28, 42, 69, 73, 26, 86, 138, 31, 47, 95, 87, 165]
    result = [len(t) for t in ex11]
    return solution == result

def grade_ex12():
    solution = [60, 63, 98, 41, 85, 39, 43, 34, 33, 133, 72, 18, 32, 58, 42, 28, 42, 69, 73, 26, 86, 138, 31, 47, 95, 87, 165]
    result = [len(t) for t in ex12(twitter_file)]
    return solution == result

def grade_ex13():
    solution = [60, 65, 37, 20, 24, 18, 63, 117, 132, 39, 73, 43, 14, 55, 33, 18, 47, 43, 92, 54, 34, 31, 97, 55, 17, 73, 126, 45, 133, 72, 39, 41, 24, 58, 23, 25, 20, 37, 31, 33, 28, 20, 24, 82, 83, 114, 134, 72, 78, 139, 101, 94, 139, 46, 31, 16, 19, 18, 22, 35, 49, 5, 55, 34, 60, 137, 40, 54, 118, 19, 57, 43]
    result = [len(t) for t in ex13(twitter_file)]
    if len(solution) <= len(result) <= len(solution) + 10: # margin of error
        return all([tweet in result for tweet in solution])
    return False

def grade_ex2():
    solution = [14, 6, 12, 5, 4, 11, 16, 14, 16, 10, 10, 8, 2, 4, 14, 11, 15, 6, 5, 8, 13, 22, 8, 5, 10, 8, 15, 16, 6, 9, 7, 7, 10, 15, 7, 9, 24, 4, 13, 3, 2, 21, 11, 17, 6, 15, 4, 19, 7, 11, 11, 6, 4, 4, 3, 2, 8, 4, 5, 17, 5, 16, 7, 6, 6, 6, 15, 11, 12, 7, 12, 7, 10, 2, 4, 10, 5, 2, 15, 5, 3, 12, 5, 18, 15, 4, 23, 8, 4, 4, 9, 11, 1, 3, 9, 5, 7, 11, 4, 12, 5, 4, 5, 5, 7, 5, 3, 20, 11, 10, 6, 12, 14, 6, 10, 7, 8, 4, 8, 9, 10, 19, 14, 7, 8, 5, 16, 15, 2, 10, 6, 16, 8, 5, 17, 23, 10, 26, 16, 31, 9, 3, 18, 23, 9, 9, 21, 12, 2, 8, 5, 2, 4, 8, 9, 6, 7, 7, 21, 3, 11, 9, 4, 10, 13, 8, 24, 5, 6, 7, 12, 8, 9, 8, 8, 8, 5, 11, 11, 17, 2, 8, 20, 8, 9, 9, 2, 5, 8, 11, 3, 13, 6, 13, 2, 16, 15, 7, 4, 8, 14, 5, 5, 4, 2, 9, 6, 25, 29, 11, 5, 18, 11, 5, 2, 9, 3, 10, 11, 19, 19, 17, 7, 17, 4, 30, 14, 26, 6, 4, 11, 12, 2, 4, 12, 12, 4, 12, 3, 10, 3, 19, 8, 8, 19, 10, 14, 25, 10, 8, 16, 12, 5, 10, 6, 5, 12, 8, 4, 3, 10, 7, 12, 22, 9, 12, 7, 5, 10, 26, 11, 6, 6, 5, 11, 4, 16, 12, 32, 11, 24, 29, 17, 12, 11, 3, 1, 2, 4, 7, 3, 7, 14, 10, 4, 11, 10, 6, 13, 9, 12, 15, 4, 3, 5, 6, 3, 12, 6, 3, 6, 8, 10, 11, 5, 4, 8, 6, 10, 4, 8, 2, 7, 3, 7, 5, 13, 7, 7, 4, 4, 4, 16, 6, 3, 4, 10, 5, 8, 7, 1, 2, 9, 3, 4, 5, 6, 8, 10, 7, 7, 5, 4, 2, 6, 21, 1, 4, 16, 4, 9, 16, 2, 3, 9, 4, 9, 10, 2, 20, 7, 6, 8, 3, 14, 4, 12, 4, 5, 12, 7, 7, 4, 10, 31, 10, 2, 6, 18, 28, 4, 14, 12, 6, 15, 18, 6, 7, 8, 9, 8, 4, 9, 32, 7, 9, 8, 5, 10, 7, 12, 12, 35, 2, 12, 2, 7, 8, 11, 5, 13, 23, 7, 13, 18, 8, 10, 13, 24, 32, 7, 15, 9, 11, 2, 4, 14, 14, 11, 3, 11, 2, 12, 6, 15, 8, 10, 10, 7, 2, 4, 4, 19, 3, 23, 17, 14, 8, 21, 10, 5, 16, 10, 12, 14, 15, 7, 13, 24, 5, 12, 36, 15, 6, 15, 19, 11, 13, 7, 11, 23, 15, 9, 10, 19, 10, 12, 29, 14, 19, 16, 22, 12, 7, 11, 26, 13, 7, 7, 9, 13, 14, 17, 8, 10, 10, 13, 21, 8, 10, 16, 17, 13, 13, 6, 10, 28, 5, 11, 10, 6, 19, 17, 22, 22, 9, 14, 9, 19, 11, 10, 15, 24, 31, 16, 6, 9, 16, 14, 10, 14, 23, 9, 17, 8, 8, 26, 13, 25, 19, 32, 8, 10, 18, 9, 13, 5, 17, 4, 18, 16, 8, 11, 9, 10, 22, 15, 13, 24, 17, 16, 18, 7, 9, 29, 16, 9, 24, 9, 11, 13, 10, 7, 4, 20, 11, 8, 27, 5, 13, 17, 12, 10, 8, 12, 4, 19, 19, 12, 10, 8, 13, 20, 12, 9, 12, 14, 10, 25, 16, 14, 7, 18, 20, 14, 22, 23, 23, 10, 12, 5, 11, 19, 14, 10, 10, 7, 23, 27, 12, 15, 20, 9, 15, 13, 16, 7, 7, 14, 18, 14, 29, 12, 13, 12, 11, 19, 8, 12, 16, 9, 7, 9, 5, 20, 10, 26, 10, 24, 7, 10, 25, 14, 10, 24, 14, 16, 29, 5, 35, 17, 12, 7, 14, 28, 7, 33, 7, 14, 18, 6, 16, 4, 9, 12, 18, 4, 11, 15, 14, 13, 19, 12, 13, 10, 14, 12, 6, 23, 22, 8, 17, 13, 14, 15, 27, 14, 10, 4, 6, 4, 5, 7, 6, 7, 10, 4, 11, 5, 8, 9, 8, 9, 6, 3, 7, 5, 8, 10, 11, 6, 2, 11, 5, 6, 7, 20, 7, 12, 19, 5, 15, 12, 6, 8, 25, 2, 13, 14, 25, 7, 12, 19, 12, 2, 10, 7, 4, 11, 9, 9, 13, 6, 15, 13, 6, 12, 10, 5, 4, 7, 8, 18, 9, 6, 7, 11, 4, 7, 9, 20, 7, 24, 9, 9, 10, 9, 30, 15, 4, 5, 5, 15, 11, 10, 24, 4, 11, 23, 16, 32, 9, 10, 13, 20, 7, 22, 21, 12, 32, 26, 24, 10, 19, 12, 17, 4, 32, 8, 12, 4, 15, 10, 23]
    result = [len(t) for t in ex2(twitter_file)]
    total = len(solution)
    correct = len([i for i in range(len(solution)) if (solution[i] -2) <= result[i] <= (solution[i] + 2)])
    return (correct / total) > 0.95

def grade_ex3():
    test_cases = [['hahahahaha', 'cooooool', '!?!?!?!!!!', '!!!!!?????']]
    res_cases = [['haha', 'cool', '!?!?!!', '!!??']]
    return res_cases == ex3(test_cases)

def grader():
    exs = [ grade_ex11, grade_ex12, grade_ex13, grade_ex2, grade_ex3]
    return 100 * len(list(filter(lambda i:i(), exs))) / len(exs)

grader()

NameError: name 'ex11' is not defined